## Distan Domain Transfer

In [4]:
import numpy as np
from keras import applications, optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import pickle
from IPython.display import clear_output
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [5]:
#Import test data from a distant domain - this dataset splits men and women
datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = datagen.flow_from_directory(
        #'tmp/test',
        'GENDER-COLOR-FERET/test',
        target_size=(150, 150),
        batch_size=1)

Found 235 images belonging to 2 classes.


### Function to build model structure

In [6]:
def getTestModel(weightFile):
    
    #We only saved the weights of our learned model. Means we need recreate the model structure first
    model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))    
    top_model = Sequential()
    top_model.add(Flatten(input_shape=model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(1, activation='sigmoid'))

    # add the model on top of the convolutional base
    test_model = Model(inputs=model.input, outputs=top_model(model.output))#model.add(top_model)
    if(weightFile is not None):    
        #use out learned weights
        test_model.load_weights(weightFile) 
    else: 
        #or the pre-trained weights from Imagenet, that wasn't touched by us
        print('Using original weights from IMAGENET')
    
    #create model
    test_model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])
    
    return test_model
          

### Function to load the model

In [8]:
def getResults(vgg16Weights,augumentation, i):    
    aug=''
    wei=''
    model=''
    if (augumentation):
        aug='Augumentation'
    else:
        aug='NoAugumentation'
        
    if vgg16Weights:
        wei='Weights'
    else:
        wei='NoWeights'
    model='vgg16_'+wei+'_'+aug+'_trainedLast_'+str(18-i)+'_Layers_.h5'
    print('Use Model: '+model)
    trainedModel=getTestModel(model) 
    prediction=trainedModel.predict_generator(test_generator,100)
    print('test accuracy: '+str(np.sum(prediction)/len(prediction)))

In [10]:
# Hop to signal from what layer a network should start train down
depth=[18, 14, 10, 6, 3, 0]

# build the VGG16 network
for i in depth:
    getResults(True,True,i)
    getResults(False,True,i)
    getResults(False,False,i)
    getResults(True,False,i)

Use Model: vgg16_Weights_Augumentation_trainedLast_0_Layers_.h5
test accuracy: 0.860477905273
Use Model: vgg16_NoWeights_Augumentation_trainedLast_0_Layers_.h5
test accuracy: 0.796162796021
Use Model: vgg16_NoWeights_NoAugumentation_trainedLast_0_Layers_.h5
test accuracy: 0.867570037842
Use Model: vgg16_Weights_NoAugumentation_trainedLast_0_Layers_.h5
test accuracy: 0.729771575928
Use Model: vgg16_Weights_Augumentation_trainedLast_4_Layers_.h5
test accuracy: 0.7996534729
Use Model: vgg16_NoWeights_Augumentation_trainedLast_4_Layers_.h5
test accuracy: 0.943097839355
Use Model: vgg16_NoWeights_NoAugumentation_trainedLast_4_Layers_.h5
test accuracy: 0.9509400177
Use Model: vgg16_Weights_NoAugumentation_trainedLast_4_Layers_.h5
test accuracy: 0.846326980591
Use Model: vgg16_Weights_Augumentation_trainedLast_8_Layers_.h5
test accuracy: 0.920284118652
Use Model: vgg16_NoWeights_Augumentation_trainedLast_8_Layers_.h5
test accuracy: 0.96036857605
Use Model: vgg16_NoWeights_NoAugumentation_trai

In [11]:
# test with imagenet weights
trainedModel=getTestModel(None) 
for i in range(1,10):
    prediction=trainedModel.predict_generator(test_generator,100)
    print('test accuracy: '+str(np.sum(prediction)/len(prediction)))

Using original weights from IMAGENET
test accuracy: 0.631822242737
test accuracy: 0.631822242737
test accuracy: 0.631822242737
test accuracy: 0.631822242737
test accuracy: 0.631822242737
test accuracy: 0.631822242737
test accuracy: 0.631822242737
test accuracy: 0.631822242737
test accuracy: 0.631822242737
